In [ ]:
#pip install opencv-python

In [ ]:
#conda install -c pytorch torchvision

In [ ]:
#pip install keras==2.1.5

In [ ]:
#pip install facenet-pytorch

In [ ]:
#conda install tensorflow

In [4]:
pip install matplotlib

  Using cached matplotlib-3.3.4-cp36-cp36m-macosx_10_9_x86_64.whl (8.5 MB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-macosx_10_9_x86_64.whl (61 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lime 0.2.0.1 requires scikit-learn>=0.18, which is not installed.
lime 0.2.0.1 requires tqdm, which is not installed.
insightface 0.1.5 requires scikit-learn, which is not installed.
insightface 0.1.5 requires tqdm, which is not installed.
aif360 0.4.0 requires scikit-learn>=0.22.1, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
from glob import glob
import cv2
import matplotlib.pylab as plt
from facenet_pytorch import MTCNN
from PIL import Image
import os
import time
import tarfile
import urllib
import urllib.request
import torch
from torch.utils.data import DataLoader
from numpy.linalg import norm

In [9]:
# change directory accordingly
CURR_DIR = os.getcwd()
print(CURR_DIR)
INPUT_DIR = CURR_DIR + "/train_sample_videos"

/Users/dugongzzz/Documents/GitHub/cmpt733-deepfake-detection


In [10]:
train_videopaths = sorted(glob(os.path.join(INPUT_DIR, "*.mp4")))[:300]
print("Number of training videos:", len(train_videopaths))

Number of training videos: 0


In [11]:
val_videopaths = sorted(glob(os.path.join(INPUT_DIR, "*.mp4")))[300:]
print("Number of validation videos:", len(val_videopaths))

Number of validation videos: 0


We want to read the json file to separate real and fake videos

In [15]:
meta = INPUT_DIR + '/metadata.json'
label_df = pd.read_json(meta)
label_df = label_df.iloc[0]
labeling_dict = label_df.to_dict()

In [28]:
def face_detector(videopaths, dir):
    startTime = time.time()

    for i in range(len(videopaths)):
        num = str(i)
        name = os.path.basename(videopaths[i])
        print(labeling_dict[name])

        width = 300
        height = 300
        dim = (width, height)

        v_cap = cv2.VideoCapture(videopaths[i])
        _, frame = v_cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
        #frame = cv2.fastNlMeansDenoisingColored(frame,None,10,10,7,21)

        new_frame = Image.fromarray(frame)

        plt.figure(figsize=(12, 8))
        plt.imshow(new_frame)
        plt.axis('off')

        mtcnn = MTCNN(select_largest=False, keep_all=True, post_process=False)#select_largest=False, device='cuda')
        test = mtcnn(new_frame)

        # detect faces in the image
        faces = mtcnn.detect(new_frame)

        if test is None:
            print("Unable to identify face")
            savepath = CURR_DIR + '/dataset/difficult_imgs/img' + num + '.jpg'
            plt.imsave(savepath, frame)
        else:   
            for each in faces[0]:
                if labeling_dict[name] == 'FAKE':
                    each1 = each.tolist()
                    x, y, w, h  = each1
                    detected_face = frame[int(y):int(h), int(x):int(w)]
                    plt.imshow(detected_face)
                    savepath = CURR_DIR + '/dataset/' + dir + '/fake/img' + num + '.jpg'
                    plt.imsave(savepath, detected_face)
                else:
                    each1 = each.tolist()
                    x, y, w, h  = each1
                    detected_face = frame[int(y):int(h), int(x):int(w)]
                    plt.imshow(detected_face)
                    savepath = CURR_DIR + '/dataset/' + dir + '/real/img' + num + '.jpg'
                    plt.imsave(savepath, detected_face)

    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))

In [2]:
#path = r'/Users/dugongzzz/Documents/GitHub/cmpt733-deepfake-detection/train_sample_videos'
videopath = '/Users/dugongzzz/Documents'
videopaths = glob(os.path.join(videopath, "wtw.mp4"))

for i in range(len(videopaths)):
    name = os.path.basename(videopaths[i])
    pre, ext = os.path.splitext(name)
    v_cap = cv2.VideoCapture(videopaths[i])
    _, frame = v_cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    #frame = cv2.fastNlMeansDenoisingColored(frame,None,10,10,7,21)

    new_frame = Image.fromarray(frame)

    plt.figure(figsize=(12, 8))
    plt.imshow(new_frame)
    plt.axis('off')

    mtcnn = MTCNN(select_largest=False, keep_all=True, post_process=False)#select_largest=False, device='cuda')
    test = mtcnn(new_frame)

    # detect faces in the image
    faces = mtcnn.detect(new_frame)

    if test is None:
        print("Unable to identify face")
    else:
        for each in faces[0]:
            each1 = each.tolist()
            x, y, w, h  = each1
            top_left = int((int(y) + int(h))/2 - 128)
            bottom_left = int((int(y) + int(h))/2 + 128)
            top_right = int((int(x) + int(w))/2 - 128)
            bottom_right = int((int(x) + int(w))/2 + 128)

            print(top_left)
            print(bottom_left)
            print(top_right)
            print(bottom_right)

            detected_face = frame[top_left:bottom_left, top_right:bottom_right]
            #detected_face = frame[bottom_left:bottom_right, top_right:bottom_right]
            savepath = '/Users/dugongzzz/Downloads/fake/' + pre + '.jpg'
            try:
                plt.imsave(savepath, detected_face)
                plt.cla()
                plt.close('all')
            except Exception:
                plt.cla()
                plt.close('all')
                pass
            

119
375
465
721
